# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.oncvpsp3.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355232590009                   -1.09    6.8    188ms
  2   -9.356717153090       -2.83       -1.42    1.5   96.7ms
  3   -9.356973272312       -3.59       -2.63    4.1    176ms
  4   -9.357012125050       -4.41       -3.17    3.4    462ms
  5   -9.357012433187       -6.51       -3.32    3.2    123ms
  6   -9.357012482992       -7.30       -3.45    1.0   82.0ms
  7   -9.357012512870       -7.52       -3.57    1.0   80.2ms
  8   -9.357012533527       -7.68       -3.74    1.0   81.1ms
  9   -9.357012544729       -7.95       -4.02    2.5    106ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.78049066809                   -0.97    7.4    571ms
  2   -18.79185491726       -1.94       -1.30    1.0    272ms
  3   -18.79230710876       -3.34       -2.80    3.8    320ms
  4   -18.79240743527       -4.00       -2.98    8.2    639ms
  5   -18.79240844230       -6.00       -3.25    1.1    221ms
  6   -18.79240856100       -6.93       -3.57    1.4    229ms
  7   -18.79240859677       -7.45       -4.26    2.5    290ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.54978194200                   -0.84    9.4    2.16s
  2   -37.56024423044       -1.98       -1.19    1.0    745ms
  3   -37.56179294578       -2.81       -2.62    5.6    1.17s
  4   -37.56451018552       -2.57       -2.42    7.6    1.79s
  5   -37.56450974018   +   -6.35       -2.42   12.2    1.41s
  6   -37.56450350681   +   -5.21       -2.41    1.0    774ms
  7   -37.56440678087   +   -4.01       -2.24    5.6    1.11s
  8   -37.56424772870   +   -3.80       -2.20    1.5    843ms
  9   -37.56429927113       -4.29       -2.28    1.0    1.06s
 10   -37.56438879453       -4.05       -2.41    1.0    751ms
 11   -37.56443407709       -4.34       -2.48    1.0    749ms
 12   -37.56447393821       -4.40       -2.57    1.0    746ms
 13   -37.56452569919       -4.29       -2.79    1.0    738ms
 14   -37.56452598365       -6.55       -2.78    1.0    702ms
 15   -37

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355362979288                   -1.09    7.0    263ms
  2   -9.356788315933       -2.85       -1.91    1.0   83.2ms
  3   -9.356985643266       -3.70       -2.59    4.8    135ms
  4   -9.356993496985       -5.10       -2.61    2.6    100ms
  5   -9.357012245788       -4.73       -3.67    1.0   63.4ms
  6   -9.357012464639       -6.66       -3.86    4.8    157ms
  7   -9.357012544681       -7.10       -4.49    1.8   78.6ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55534136730                   -0.84    8.9    1.94s
  2   -37.52864138547   +   -1.57       -1.55    1.0    627ms
  3   -7.894256030492   +    1.47       -0.33    8.5    1.72s
  4   -37.54932420222        1.47       -1.81    8.2    1.98s
  5   -37.56006308788       -1.97       -1.89    2.9    998ms
  6   -37.55593742763   +   -2.38       -2.06    2.6    832ms
  7   -37.54729640861   +   -2.06       -1.91    3.8    975ms
  8   -37.46331289404   +   -1.08       -1.56    5.0    1.15s
  9   -37.56342867801       -1.00       -2.36    5.1    1.41s
 10   -37.56438472513       -3.02       -2.65    2.4    902ms
 11   -37.56452227726       -3.86       -3.07    1.9    827ms
 12   -37.56454690560       -4.61       -3.30    3.1    998ms
 13   -37.56455284890       -5.23       -3.72    3.0    918ms
 14   -37.56455385662       -6.00       -4.07    4.1    1.32s


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.